In [39]:
import numpy as np
import matplotlib.pyplot as plt
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, Aer, IBMQ
from qiskit import *
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from qiskit.tools.visualization import circuit_drawer

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2021-04-11 08:00:26,427: Credentials are already in use. The existing account in the session will be replaced.


In [93]:
class UserInteraction():
    def __init__(self):
        self.str_q0 = "q0 -"
        self.str_q1 = "q1 -"
        self.wire = "-----------"
        
        #Initial position in grid
        self.i, self.j = [0,0]
        #Creates grid
        self.grid=np.matrix([[2,0,3,0,0],
                            [1,0,1,0,1],
                            [1,0,0,0,1],
                            [1,1,1,1,1]])
    
    def apply_gates(self):
        print("Insert gates you want to apply (Ex: H-X-H)")
        q0 = input("q0--")
        q1 = input("q1--")
        
        self.gates_q0 = []
        self.gates_q1 = []
        for gate in q0:
            if gate != "-":
                self.gates_q0.append(gate)
        
        for gate in q1:
            if gate != "-":
                self.gates_q1.append(gate)
        
        self.generate_circuit(self.gates_q0,self.gates_q1)
        
        
    def generate_circuit(self,list0, list1):
        circ = QuantumCircuit(2)
        
        #gates q0
        for gate in list0:
            if gate == 'X':
                circ.x(0)
            elif gate == 'H':
                circ.h(0)
        
        #gates q1
        for gate in list1:
            if gate == 'X':
                circ.x(1)
            elif gate == 'H':
                circ.h(1)
        
        #Creating the measurement
        meas = QuantumCircuit(2,2)
        meas.barrier(range(2))
        meas.measure(range(2), range(2))
        
        
        self.qc = circ+meas
        print(self.qc)
    
    def movement(self):
        
        
        #Measurement
        simulator = BasicAer.get_backend('qasm_simulator')
        compiled_circuit = transpile(self.qc, simulator)
        
        #initialize dictonary
        counts= {
          "00": 0,
          "01": 0,
          "10": 0,
          "11": 0
        }

        # Execute the circuit on the qasm simulator
        job = simulator.run(compiled_circuit, shots=1000)

        # Grab results from the job
        result = job.result()

        # Returns counts
        self.counts = result.get_counts(self.qc)
        
        
        #Movement
        max_counts=900

        #Makes sure to get every state in the dictionary (even if it has 0 counts)
        self.counts['00']=self.counts.get('00', 0)
        self.counts['01']=self.counts.get('01', 0)
        self.counts['10']=self.counts.get('10', 0)
        self.counts['11']=self.counts.get('11', 0)

        # 00: DOWN
        if self.counts["00"]>max_counts:
            if self.grid[self.i+1,self.j]==0:
                self.grid[self.i,self.j]=0
                self.grid[(self.i+1),self.j]=2
                self.i+=1
            else:
                print('Impossible movement. Try again')

        #01: UP
        elif self.counts["01"]>max_counts:
            if self.grid[self.i-1,self.j]==0:
                self.grid[self.i,self.j]=0
                self.grid[self.i-1,self.j]=2
                self.i += -1
            else:
                print('Impossible movement. Try again')

        #10: RIGHT        
        elif self.counts["10"]>max_counts:
            if self.grid[self.i,self.j+1]==0:
                self.grid[self.i,self.j]=0
                self.grid[self.i,self.j+1]=2
                self.j += 1
            else:
                print('Impossible movement. Try again')

        #11: LEFT
        elif self.counts["11"]>max_counts:
            if self.grid[self.i,self.j-1]==0:
                self.grid[self.i,self.j]=0
                self.grid[self.i,self.j-1]=2
                j += -1
            else:
                print('Impossible movement. Try again')

        return self.grid


In [99]:
A = UserInteraction()
print('Initial position:\n{}'.format(A.grid))


while A.grid[0,4] != 2:
    A.apply_gates()
    print(A.movement())
print('Congratulations!!')

Initial position:
[[2 0 0 1 1]
 [1 1 0 0 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
Insert gates you want to apply (Ex: H-X-H)
q0--
q1--X
           ░ ┌─┐   
q_0: ──────░─┤M├───
     ┌───┐ ░ └╥┘┌─┐
q_1: ┤ X ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 
[[0 2 0 1 1]
 [1 1 0 0 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
Insert gates you want to apply (Ex: H-X-H)
q0--
q1--X
           ░ ┌─┐   
q_0: ──────░─┤M├───
     ┌───┐ ░ └╥┘┌─┐
q_1: ┤ X ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 
[[0 0 2 1 1]
 [1 1 0 0 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
Insert gates you want to apply (Ex: H-X-H)
q0--
q1--
      ░ ┌─┐   
q_0: ─░─┤M├───
      ░ └╥┘┌─┐
q_1: ─░──╫─┤M├
      ░  ║ └╥┘
c: 2/════╩══╩═
         0  1 
[[0 0 0 1 1]
 [1 1 2 0 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
Insert gates you want to apply (Ex: H-X-H)
q0--
q1--X
           ░ ┌─┐   
q_0: ──────░─┤M├───
     ┌───┐ ░ └╥┘┌─┐
q_1: ┤ X ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 
[[0 0 0 1 1]
 [1 1 0 2 1]
 [1 

Insert gates you want to apply (Ex: H-X-H)
q0--
q1--X
           ░ ┌─┐   
q_0: ──────░─┤M├───
     ┌───┐ ░ └╥┘┌─┐
q_1: ┤ X ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 
[[0 2 0 1 1]
 [1 1 0 0 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
{'10': 1000, '00': 0, '01': 0, '11': 0}


Insert gates you want to apply (Ex: H-X-H)
q0--
q1--X
           ░ ┌─┐   
q_0: ──────░─┤M├───
     ┌───┐ ░ └╥┘┌─┐
q_1: ┤ X ├─░──╫─┤M├
     └───┘ ░  ║ └╥┘
c: 2/═════════╩══╩═
              0  1 
[[0 0 0 1 1]
 [1 1 0 2 1]
 [1 1 1 0 1]
 [1 1 1 0 0]]
{'10': 1000, '00': 0, '01': 0, '11': 0}
